In [1]:
import keras
import os
import pandas as pd
import numpy as np
import urllib
import random
import os

from keras.layers.core import Dense, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer

from nltk import confusionmatrix
from sklearn import metrics

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [2]:
# Download annotated comments and annotations. 
# If you're Tracy, Courtney, or Amandalynne, don't run this step 
# because you already have the data! If you aren't us, you will 
# probably need to do this step. 
# It will take a while. 
ANNOTATED_COMMENTS_URL = 'https://ndownloader.figshare.com/files/7038044' 
ANNOTATIONS_URL = 'https://ndownloader.figshare.com/files/7383751' 


def download_file(url, fname):
    urllib.request.urlretrieve(url, fname)

                
download_file(ANNOTATED_COMMENTS_URL, 'attack_annotated_comments.tsv')
download_file(ANNOTATIONS_URL, 'attack_annotations.tsv')

In [2]:
print(os.getcwd())

C:\Users\Courtney\Desktop\Trolling_RNN


In [4]:
# Read the data into a Pandas dataframe.
comments = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('attack_annotations.tsv',  sep = '\t')

# Label a comment as an attack if over half of annotators did so.
# We can tinker with this threshold later.
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

# Join labels and comments
comments['attack'] = labels

# Preprocess the data -- remove newlines, tabs, quotes
# Something to consider: remove Wikipedia style markup (::'s and =='s)
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("`", " "))

In [5]:
# Grab the training, dev, and test data (3:1:1 split)
train_data = comments.loc[comments['split'] == 'train']
dev_data = comments.loc[comments['split'] == 'dev']
test_data = comments.loc[comments['split'] == 'test']


In [6]:
# The list of gold-standard labels for the data
train_labels = train_data["attack"].tolist()
dev_labels = dev_data["attack"].tolist()
test_labels = test_data["attack"].tolist()
# Put all the training data (comments) into a list
train_texts = train_data["comment"].tolist()
dev_texts = dev_data["comment"].tolist()
test_texts = test_data["comment"].tolist()

In [7]:
def randomly_sample(comments, labels):
    # grab all the attacks to see how many we need to match
    attack_indices = [i for i in range(len(comments)) if labels[i] == True]
    new_training = [comments[i] for i in attack_indices]
    new_labels = [labels[i] for i in attack_indices]
    
    # grab all the ones that are not attacks, shuffle them and select the same number of non-attacks
    non_attack_indices = [i for i in range(len(comments)) if labels[i] == False]
    random.shuffle(non_attack_indices)
    for i in range(len(attack_indices)):
        new_training.append(comments[i])
        new_labels.append(labels[i])
        
    # shuffle the training and labeling so that they're still matched 1:1
    shuf_indices = [i for i in range(len(new_training))]
    random.shuffle(shuf_indices)
    shuffled_training = [new_training[i] for i in shuf_indices]
    shuffled_labels = [new_labels[i] for i in shuf_indices]
    
    return shuffled_training, shuffled_labels
    

In [8]:
# randomly select non-attack data such that it creates an even split with the attack data
train_texts, train_labels = randomly_sample(train_texts, train_labels)
dev_texts, dev_labels = randomly_sample(dev_texts, dev_labels)
test_texts, test_labels = randomly_sample(test_texts, test_labels)
print(len(train_texts), len(train_labels))

16158 16158


In [9]:
# The word level tokenizer using only top 5000 words
tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)

In [10]:
# Fit it to the training data.
tokenizer.fit_on_texts(train_texts)

In [11]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

45816


In [12]:
dev_tokenizer =  keras.preprocessing.text.Tokenizer(num_words=5000)
dev_tokenizer.fit_on_texts(dev_texts)

test_tokenizer =  keras.preprocessing.text.Tokenizer(num_words=5000)
test_tokenizer.fit_on_texts(test_texts)

In [13]:
# Transform each comment in the training data to arrays of equal length
# I should really make this tokenization step into a function! To-do for AP.
train_matrix = tokenizer.texts_to_matrix(train_texts)
dev_matrix = tokenizer.texts_to_matrix(dev_texts)
test_matrix = tokenizer.texts_to_matrix(test_texts)

In [18]:
# Dimensions of our training matrix
train_matrix.shape[0], train_matrix.shape[1]

(16158, 5000)

In [19]:
# create an embedding layer of wordvec values
# assumes you have downloaded he glove.6b dataset here: http://nlp.stanford.edu/data/glove.6B.zip
embeddings_index = {}
f = open(os.path.join(str(os.getcwd())+"/glove.6B", 'glove.6B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Found 400000 word vectors.


In [20]:
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
# Create custom skipgram embedding layer

embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=train_matrix.shape[1],
                            trainable=False)

In [23]:
# Make a model.
model = Sequential()
# Add embedding layer.
# Recall from earlier that char unigram vocab size is 1557.
# Train_matrix shape is 69526 x 126531, use that as input dimension.
model.add(embedding_layer)
model.add(Flatten())
# Commented out for now bc... the example I was following only had one Dense layer
#model.add(Dense(2, input_dim=len(char_unigrams)))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
# Train model
model.fit(train_matrix, train_labels,
          batch_size=100,
          epochs=4)

Epoch 1/4
16158/16158 [==============================] - 78s - loss: 0.3710 - acc: 0.8468      

In [26]:

# Evaluate model
score, acc = model.evaluate(test_matrix, test_labels, batch_size=128)
print('\nAccuracy: %1.4f' % acc)
print('Score: %1.4f' % score)

5504/5512 [============================>.] - ETA: 0s 

In [ ]:
# Definitions for AUC and F1 evaluation
def pred_to_labels(predictions):
    result = np.hstack((predictions)).tolist()
    return [result[i] >= 0.5 for i in range(len(result))]
    

def f1_score(y_true, y_pred):
    y_pred = pred_to_labels(y_pred)
    return metrics.f1_score(y_true, y_pred)
    

def auc_score(y_true, y_pred):
    return metrics.roc_auc_score(y_true, y_pred)

In [ ]:
# Get F1 and AUROC values
y_pred = model.predict(test_matrix, batch_size=32, verbose=0)
f1 = f1_score(test_labels, y_pred)
roc = auc_score(test_labels, y_pred)

print('F1:{}'.format(f1))
print('AUROC:{}'.format(roc))

In [ ]:
# Get confusion matrix
cm = confusionmatrix.ConfusionMatrix(test_labels, pred_to_labels(y_pred))
print(cm.pretty_format(sort_by_count=True, show_percents=True, truncate=9))